In [1]:
pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.4/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install imbalanced-learn


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.4/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('logistic_regression.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396030 entries, 0 to 396029
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   loan_amnt             396030 non-null  float64
 1   term                  396030 non-null  object 
 2   int_rate              396030 non-null  float64
 3   installment           396030 non-null  float64
 4   grade                 396030 non-null  object 
 5   sub_grade             396030 non-null  object 
 6   emp_title             373103 non-null  object 
 7   emp_length            377729 non-null  object 
 8   home_ownership        396030 non-null  object 
 9   annual_inc            396030 non-null  float64
 10  verification_status   396030 non-null  object 
 11  issue_d               396030 non-null  object 
 12  loan_status           396030 non-null  object 
 13  purpose               396030 non-null  object 
 14  title                 394274 non-null  object 
 15  

### Problem Statement

LoanTap seeks to develop an underwriting system to assess the creditworthiness of individuals applying for a **Personal Loan**. Given a set of applicant attributes (demographics, employment, financial history, and credit behavior), the system should address two key objectives:
1. **Loan Decisioning** – Determine whether the loan application should be **approved or rejected** based on the applicant’s creditworthiness.  
2. **Repayment Recommendation** – For approved applicants, recommend **suitable loan terms** (loan amount, tenure, and interest rate) that balance:  
   - **Customer affordability**  
   - **LoanTap’s risk management objectives**  


In [6]:
df.shape

(396030, 27)

In [7]:
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,10000.0,36 months,11.44,329.48,B,B4,Marketing,10+ years,RENT,117000.0,...,16.0,0.0,36369.0,41.8,25.0,w,INDIVIDUAL,0.0,0.0,"0174 Michelle Gateway\r\nMendozaberg, OK 22690"
1,8000.0,36 months,11.99,265.68,B,B5,Credit analyst,4 years,MORTGAGE,65000.0,...,17.0,0.0,20131.0,53.3,27.0,f,INDIVIDUAL,3.0,0.0,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113"
2,15600.0,36 months,10.49,506.97,B,B3,Statistician,< 1 year,RENT,43057.0,...,13.0,0.0,11987.0,92.2,26.0,f,INDIVIDUAL,0.0,0.0,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113"
3,7200.0,36 months,6.49,220.65,A,A2,Client Advocate,6 years,RENT,54000.0,...,6.0,0.0,5472.0,21.5,13.0,f,INDIVIDUAL,0.0,0.0,"823 Reid Ford\r\nDelacruzside, MA 00813"
4,24375.0,60 months,17.27,609.33,C,C5,Destiny Management Inc.,9 years,MORTGAGE,55000.0,...,13.0,0.0,24584.0,69.8,43.0,f,INDIVIDUAL,1.0,0.0,"679 Luna Roads\r\nGreggshire, VA 11650"


In [8]:
df.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,mort_acc,pub_rec_bankruptcies
count,396030.000000,396030.000000,396030.000000,3.960300e+05,396030.000000,396030.000000,396030.000000,3.960300e+05,395754.000000,396030.000000,358235.000000,395495.000000
mean,14113.888089,13.639400,431.849698,7.420318e+04,17.379514,11.311153,0.178191,1.584454e+04,53.791749,25.414744,1.813991,0.121648
std,8357.441341,4.472157,250.727790,6.163762e+04,18.019092,5.137649,0.530671,2.059184e+04,24.452193,11.886991,2.147930,0.356174
min,500.000000,5.320000,16.080000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,2.000000,0.000000,0.000000
25%,8000.000000,10.490000,250.330000,4.500000e+04,11.280000,8.000000,0.000000,6.025000e+03,35.800000,17.000000,0.000000,0.000000
50%,12000.000000,13.330000,375.430000,6.400000e+04,16.910000,10.000000,0.000000,1.118100e+04,54.800000,24.000000,1.000000,0.000000
75%,20000.000000,16.490000,567.300000,9.000000e+04,22.980000,14.000000,0.000000,1.962000e+04,72.900000,32.000000,3.000000,0.000000
max,40000.000000,30.990000,1533.810000,8.706582e+06,9999.000000,90.000000,86.000000,1.743266e+06,892.300000,151.000000,34.000000,8.000000


#### ==========================
### Data Analysis
#### ==========================

#### Shape of Data

In [9]:
print("Shape of dataset:", df.shape)

Shape of dataset: (396030, 27)


#### Data Types

In [10]:
print("\nData Types:\n", df.dtypes)


Data Types:
 loan_amnt               float64
term                     object
int_rate                float64
installment             float64
grade                    object
sub_grade                object
emp_title                object
emp_length               object
home_ownership           object
annual_inc              float64
verification_status      object
issue_d                  object
loan_status              object
purpose                  object
title                    object
dti                     float64
earliest_cr_line         object
open_acc                float64
pub_rec                 float64
revol_bal               float64
revol_util              float64
total_acc               float64
initial_list_status      object
application_type         object
mort_acc                float64
pub_rec_bankruptcies    float64
address                  object
dtype: object


#### Missing Values

In [11]:
print("\nMissing Values:\n", df.isnull().sum())


Missing Values:
 loan_amnt                   0
term                        0
int_rate                    0
installment                 0
grade                       0
sub_grade                   0
emp_title               22927
emp_length              18301
home_ownership              0
annual_inc                  0
verification_status         0
issue_d                     0
loan_status                 0
purpose                     0
title                    1756
dti                         0
earliest_cr_line            0
open_acc                    0
pub_rec                     0
revol_bal                   0
revol_util                276
total_acc                   0
initial_list_status         0
application_type            0
mort_acc                37795
pub_rec_bankruptcies      535
address                     0
dtype: int64


In [12]:
# Convert Categorical Columns to 'category'
categorical_cols = df.select_dtypes(include=['object']).columns
# Drop date-like fields from categorical
categorical_cols = [col for col in categorical_cols if col not in ['issue_d', 'earliest_cr_line','address','emp_length']]

for col in categorical_cols:
    df[col] = df[col].astype('category')

print("\nCategorical columns converted to 'category':", categorical_cols)


Categorical columns converted to 'category': ['term', 'grade', 'sub_grade', 'emp_title', 'home_ownership', 'verification_status', 'loan_status', 'purpose', 'title', 'initial_list_status', 'application_type']


#### Statistical Summary (Numerical Attributes)

In [13]:
# ======================

# ======================
print("\nStatistical Summary:\n")
df.describe()


Statistical Summary:



,loan_amnt,int_rate,installment,annual_inc,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,mort_acc,pub_rec_bankruptcies
count,396030.000000,396030.000000,396030.000000,3.960300e+05,396030.000000,396030.000000,396030.000000,3.960300e+05,395754.000000,396030.000000,358235.000000,395495.000000
mean,14113.888089,13.639400,431.849698,7.420318e+04,17.379514,11.311153,0.178191,1.584454e+04,53.791749,25.414744,1.813991,0.121648
std,8357.441341,4.472157,250.727790,6.163762e+04,18.019092,5.137649,0.530671,2.059184e+04,24.452193,11.886991,2.147930,0.356174
min,500.000000,5.320000,16.080000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,2.000000,0.000000,0.000000
25%,8000.000000,10.490000,250.330000,4.500000e+04,11.280000,8.000000,0.000000,6.025000e+03,35.800000,17.000000,0.000000,0.000000
50%,12000.000000,13.330000,375.430000,6.400000e+04,16.910000,10.000000,0.000000,1.118100e+04,54.800000,24.000000,1.000000,0.000000
75%,20000.000000,16.490000,567.300000,9.000000e+04,22.980000,14.000000,0.000000,1.962000e+04,72.900000,32.000000,3.000000,0.000000
max,40000.000000,30.990000,1533.810000,8.706582e+06,9999.000000,90.000000,86.000000,1.743266e+06,892.300000,151.000000,34.000000,8.000000


#### Missing Value Treatment

In [14]:
print("\nMissing Values:\n",)
df.isnull().sum()[df.isnull().sum() > 0].reset_index()


Missing Values:



,index,0
0,emp_title,22927
1,emp_length,18301
2,title,1756
3,revol_util,276
4,mort_acc,37795
5,pub_rec_bankruptcies,535


In [15]:
df.dropna(inplace=True)

In [16]:
print("\nMissing Values:\n",)
df.isnull().sum()[df.isnull().sum() > 0].reset_index()


Missing Values:



,index,0


In [17]:
df = df.drop(columns=['title'])

#### Outlier Treatment

In [18]:
import pandas as pd

# Columns to check for outliers
num_cols = ['open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'pub_rec_bankruptcies', 'dti', 'loan_amnt', 'installment', 'annual_inc']

# Copy the dataframe
outlier_removed_df = df.copy()

# Remove outliers using IQR method
for col in num_cols:
    Q1 = outlier_removed_df[col].quantile(0.25)
    Q3 = outlier_removed_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Keep only rows within the IQR bounds
    outlier_removed_df = outlier_removed_df[(outlier_removed_df[col] >= lower_bound) & (outlier_removed_df[col] <= upper_bound)]

# Check the resulting dataframe
outlier_removed_df.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,mort_acc,pub_rec_bankruptcies
count,241325.000000,241325.000000,241325.000000,241325.000000,241325.000000,241325.000000,241325.0,241325.000000,241325.000000,241325.000000,241325.000000,241325.0
mean,13580.606133,13.712532,413.868354,65314.203010,17.891109,10.871037,0.0,13397.638666,55.218938,23.755143,1.573223,0.0
std,7464.375919,4.503506,212.402733,26981.254017,8.008883,4.285470,0.0,9121.645679,23.964014,10.057894,1.980907,0.0
min,1000.000000,5.320000,28.750000,5000.000000,0.000000,1.000000,0.0,0.000000,0.000000,2.000000,0.000000,0.0
25%,8000.000000,10.490000,257.310000,45000.000000,11.910000,8.000000,0.0,6416.000000,38.000000,16.000000,0.000000,0.0
50%,12000.000000,13.350000,375.490000,60000.000000,17.480000,10.000000,0.0,11337.000000,56.300000,23.000000,1.000000,0.0
75%,18225.000000,16.550000,545.310000,81900.000000,23.550000,14.000000,0.0,18503.000000,73.900000,30.000000,3.000000,0.0
max,38000.000000,30.990000,1045.260000,145000.000000,40.640000,23.000000,0.0,42572.000000,892.300000,52.000000,32.000000,0.0


#### Feature Engineering

In [19]:
import pandas as pd

def binarize_features(df, cols):
    df_copy = df.copy()
    for col in cols:
        df_copy[col] = df_copy[col].apply(lambda x: 0 if x == 0 else 1)
    return df_copy

binary_cols = ['pub_rec', 'mort_acc', 'pub_rec_bankruptcies']

binarized_df = binarize_features(outlier_removed_df, binary_cols)

In [20]:
import re
def categorize_job(title):
    if pd.isnull(title) or str(title).strip() == '':
        return "Unknown"
    
    title = str(title).lower()

    # Define keyword-based rules with expanded coverage
    if re.search(r"teacher|professor|instructor|principal|educator|tutor", title):
        return "Education"
    elif re.search(r"nurse|rn|lpn|cna|healthcare|physician|doctor|pharmacist|medical|clinic|hospital", title):
        return "Healthcare"
    elif re.search(r"driver|truck|transport|operator|delivery|chauffeur", title):
        return "Transport"
    elif re.search(r"engineer|developer|programmer|coder|software|technician|network|it|data|systems", title):
        return "Technology"
    elif re.search(r"manager|supervisor|director|vp|president|executive|superintendent|controller|coordinator|administrator|lead", title):
        return "Management"
    elif re.search(r"accountant|auditor|finance|analyst|advisor|bookkeeper|underwriter|controller|cfo|financial|treasurer", title):
        return "Finance"
    elif re.search(r"sales|marketing|customer|associate|representative|clerk|cashier|retail|merchandiser|store", title):
        return "Sales/Marketing"
    elif re.search(r"legal|attorney|paralegal|lawyer|counsel|judge", title):
        return "Legal"
    elif re.search(r"military|us army|usaf|navy|air force|marine|coast guard|soldier|veteran", title):
        return "Military"
    elif re.search(r"police|officer|firefighter|security|sheriff|corrections", title):
        return "Public Safety"
    elif re.search(r"mechanic|electrician|machinist|construction|foreman|plumber|welder|carpenter|technician|hvac", title):
        return "Skilled Trade"
    elif re.search(r"real estate|realtor|broker|property|leasing", title):
        return "Real Estate"
    elif re.search(r"bank|loan|mortgage|credit union", title):
        return "Banking"
    elif re.search(r"hr|human resources|recruiter|talent|staffing", title):
        return "HR/Recruitment"
    elif re.search(r"server|waiter|waitress|bartender|cook|chef|hospitality|hotel|restaurant", title):
        return "Hospitality"
    elif re.search(r"government|federal|state|county|city|public works|council", title):
        return "Government"
    else:
        return "Other"

engg_df = binarized_df.copy()
# Apply function
engg_df['emp_category'] = engg_df['emp_title'].apply(categorize_job)

# Check distribution
print(engg_df['emp_category'].value_counts())

emp_category
Other              94466
Management         43728
Technology         32073
Healthcare         17088
Sales/Marketing    10579
Transport           7853
Finance             7387
Education           6780
Public Safety       4717
Skilled Trade       4100
Banking             2980
Government          2874
Hospitality         2079
Legal               1758
Military            1378
HR/Recruitment       969
Real Estate          516
Name: count, dtype: int64


In [21]:
engg_df['emp_length'].value_counts().reset_index()

emp_length_map = {
    '10+ years': 10,
    '9 years': 9,
    '8 years': 8,
    '7 years': 7,
    '6 years': 6,
    '5 years': 5,
    '4 years': 4,
    '3 years': 3,
    '2 years': 2,
    '1 year': 1,
    '< 1 year': 0
}
# Replace with mapping
engg_df['emp_length'] = engg_df['emp_length'].map(emp_length_map)

In [22]:
engg_df.describe(include=[object, 'category'])

,term,grade,sub_grade,emp_title,home_ownership,verification_status,issue_d,loan_status,purpose,earliest_cr_line,initial_list_status,application_type,address,emp_category
count,241325,241325,241325,241325,241325,241325,241325,241325,241325,241325,241325,241325,241325,241325
unique,2,7,35,117364,6,3,58,2,13,631,2,3,240367,17
top,36 months,B,B3,Teacher,MORTGAGE,Not Verified,Oct-2014,Fully Paid,debt_consolidation,Oct-2001,f,INDIVIDUAL,USS Smith\r\nFPO AP 70466,Other
freq,183540,71917,16544,3244,116029,86632,9504,193289,147442,1893,138527,241107,7,94466


#### Univariate Analysis

In [23]:
engg_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 241325 entries, 0 to 396028
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   loan_amnt             241325 non-null  float64 
 1   term                  241325 non-null  category
 2   int_rate              241325 non-null  float64 
 3   installment           241325 non-null  float64 
 4   grade                 241325 non-null  category
 5   sub_grade             241325 non-null  category
 6   emp_title             241325 non-null  category
 7   emp_length            241325 non-null  int64   
 8   home_ownership        241325 non-null  category
 9   annual_inc            241325 non-null  float64 
 10  verification_status   241325 non-null  category
 11  issue_d               241325 non-null  object  
 12  loan_status           241325 non-null  category
 13  purpose               241325 non-null  category
 14  dti                   241325 non-null  fl

In [24]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# sns.set(style="whitegrid", palette="muted")
# plt.rcParams["figure.figsize"] = (10, 5)
# plt.rcParams['font.family'] = 'DejaVu Sans' 
# plt.rcParams['text.usetex'] = False
# plt.rcParams['axes.unicode_minus'] = False

# # Continuous Variables
# num_cols = engg_df.select_dtypes(include=['float64', 'int64']).columns

# for col in num_cols:
#     plt.figure()
#     sns.histplot(engg_df[col].dropna(), kde=True, bins=30)
#     plt.title(f"Distribution of {col}")
#     plt.xlabel(col)
#     plt.ylabel("Frequency")
#     plt.show()

# categorical_cols = ['grade','sub_grade','home_ownership','verification_status','purpose','emp_category','initial_list_status']

# # Categorical Variables
# for col in categorical_cols:
#     plt.figure()
#     sns.countplot(data=engg_df, x=col, order=engg_df[col].value_counts().index)
#     plt.title(f"Countplot of {col}")
#     plt.xticks(rotation=45)
#     plt.show()

#### Bivariate Analysis

In [25]:
# target = 'loan_status'

# # Numerical vs Target
# for col in num_cols:
#     plt.figure()
#     sns.boxplot(data=engg_df, x=target, y=col)
#     plt.title(f"{col} vs {target}")
#     plt.show()

# # Categorical vs Target
# for col in categorical_cols:
#     plt.figure()
#     sns.countplot(data=engg_df, x=col, hue=target, order=engg_df[col].value_counts().index)
#     plt.title(f"{col} vs {target}")
#     plt.xticks(rotation=45)
#     plt.show()


#### Data Preparation for Model Training

In [26]:
engg_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 241325 entries, 0 to 396028
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   loan_amnt             241325 non-null  float64 
 1   term                  241325 non-null  category
 2   int_rate              241325 non-null  float64 
 3   installment           241325 non-null  float64 
 4   grade                 241325 non-null  category
 5   sub_grade             241325 non-null  category
 6   emp_title             241325 non-null  category
 7   emp_length            241325 non-null  int64   
 8   home_ownership        241325 non-null  category
 9   annual_inc            241325 non-null  float64 
 10  verification_status   241325 non-null  category
 11  issue_d               241325 non-null  object  
 12  loan_status           241325 non-null  category
 13  purpose               241325 non-null  category
 14  dti                   241325 non-null  fl

In [27]:
# Select categorical columns
categorical_cols = engg_df.select_dtypes(include=['object', 'category']).columns

for col in categorical_cols:
    unique_vals = engg_df[col].unique()
    print(f"Column: {col}")
    print(f"Number of unique values: {len(unique_vals)}")
    print(f"Unique values (sample up to 20): {unique_vals[:20]}\n")

Column: term
Number of unique values: 2
Unique values (sample up to 20): [' 36 months', ' 60 months']
Categories (2, object): [' 36 months', ' 60 months']

Column: grade
Number of unique values: 7
Unique values (sample up to 20): ['B', 'A', 'C', 'E', 'D', 'F', 'G']
Categories (7, object): ['A', 'B', 'C', 'D', 'E', 'F', 'G']

Column: sub_grade
Number of unique values: 35
Unique values (sample up to 20): ['B4', 'B5', 'B3', 'A2', 'C5', ..., 'C1', 'D5', 'D2', 'E1', 'E2']
Length: 20
Categories (35, object): ['A1', 'A2', 'A3', 'A4', ..., 'G2', 'G3', 'G4', 'G5']

Column: emp_title
Number of unique values: 117364
Unique values (sample up to 20): ['Marketing', 'Credit analyst ', 'Statistician', 'Client Advocate', 'Destiny Management Inc.', ..., 'Personal Trainer', 'Francis Howell School District', 'Office Manager', 'Operations Manager', 'Bartender']
Length: 20
Categories (173105, object): ['       NSA Industries llc', '   Fibro Source', '   Long Ilsand College Hospital', '   mortgage banker', .

In [28]:
engg_df['term'] = engg_df['term'].map({' 36 months': 36, ' 60 months': 60})

In [29]:
# engg_df['state'] = engg_df['address'].apply(lambda x: x.split()[-2] if pd.notnull(x) else 'Unknown')
import re

def extract_zip(address):
    # Find the last 5 consecutive digits in the string
    match = re.search(r'(\d{5})(?!.*\d)', address)
    return match.group(1) if match else None

# Apply to your DataFrame column (say 'address')
engg_df['zipcode'] = engg_df['address'].apply(extract_zip)

# # Check result
# print(engg_df[['address', 'zipcode']].head(20))

In [30]:
categorical_cols = ['purpose', 'verification_status', 
                    'home_ownership', 'grade','zipcode']
# 'sub_grade','emp_category','state'removed for now
# Create dummies for all columns at once, drop first category, and ensure 0/1
engg_df = pd.get_dummies(engg_df, columns=categorical_cols, drop_first=True)

# Convert any boolean columns (True/False) to 0/1
bool_cols = engg_df.select_dtypes(include='bool').columns

print(bool_cols)


Index(['purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_educational', 'purpose_home_improvement', 'purpose_house',
       'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
       'purpose_other', 'purpose_renewable_energy',
       ...
       'zipcode_99962', 'zipcode_99968', 'zipcode_99972', 'zipcode_99973',
       'zipcode_99976', 'zipcode_99978', 'zipcode_99983', 'zipcode_99991',
       'zipcode_99995', 'zipcode_99996'],
      dtype='object', length=30215)


In [ ]:
engg_df[bool_cols] = engg_df[bool_cols].astype(int)

In [ ]:
engg_df['initial_list_status'] = engg_df['initial_list_status'].map({'f': 0, 'w': 1},)

In [ ]:
# mapping = {'INDIVIDUAL': 0, 'JOINT': 1, 'DIRECT_PAY': 2}
# outlier_trtd_df['application_type_encoded'] = outlier_trtd_df['application_type'].map(mapping)

In [ ]:
# Convert earliest credit line to datetime (Removed for now)
# engg_df['earliest_cr_line'] = pd.to_datetime(engg_df['earliest_cr_line'], format='%b-%Y', errors='coerce')

# # Create credit history in years (continuous)
# engg_df['credit_history_years'] = (pd.to_datetime('today') - engg_df['earliest_cr_line']).dt.days / 365

In [ ]:
encoded_df = engg_df.drop(columns=['earliest_cr_line','address','application_type','issue_d','emp_title','installment','sub_grade','emp_category','emp_length'])

In [ ]:
pd.options.display.max_columns = None
encoded_df.head()

In [ ]:
# bool_cols = encoded_df.select_dtypes(include='bool').columns
# print(bool_cols)
# encoded_df[bool_cols] = encoded_df[bool_cols].astype(int)
# encoded_df[bool_cols].head()

In [ ]:
encoded_df['loan_status'] = encoded_df['loan_status'].map({'Charged Off': 1, 'Fully Paid': 0})

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = encoded_df.drop(columns=['loan_status'])  # Features
y = encoded_df['loan_status']                 # Target

feature_names = X.columns 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# string_cols = X_train.select_dtypes(include=['object', 'string']).columns

# print("String columns in X_train:")
# print(list(string_cols))

In [ ]:
X_train = pd.DataFrame(X_train,columns=X.columns )
X_train.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
# ---------------------
# 4. Logistic Regression
# ---------------------
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# ---------------------
# 5. Evaluate
# ---------------------
y_pred = clf.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# import pandas as pd
# import numpy as np

# # Get feature importance from logistic regression
# feature_importance = pd.DataFrame({
#     'Feature': feature_names,
#     'Coefficient': log_reg.coef_[0],
#     'Abs_Coefficient': np.abs(log_reg.coef_[0])
# }).sort_values(by='Abs_Coefficient', ascending=False)

# print(feature_importance.head(20))  # Top 20 most important features

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Basic SMOTE (works when all features are numeric)
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print("Before SMOTE:\n", y_train.value_counts())
print("After SMOTE:\n", y_train_res.value_counts())

# Train Logistic Regression on resampled data
log_reg = LogisticRegression(class_weight="balanced")
log_reg.fit(X_train_res, y_train_res)

# Predictions
y_pred = log_reg.predict(X_test)

# Evaluation
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# # Check for NaN values column-wise
# nan_counts = pd.DataFrame({
#     "missing_count": X_train_res.isna().sum(),
#     "missing_ratio": X_train_res.isna().mean()
# })

# print("Total NaN values in resampled data:", X_train_res.isna().sum().sum())
# print(nan_counts[nan_counts["missing_count"] > 0])

In [ ]:
# # Evaluation
# cm = confusion_matrix(y_test, y_pred)
# print("Confusion Matrix:\n", cm)
# print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# from sklearn.metrics import (
#     roc_curve, roc_auc_score,
#     precision_recall_curve, average_precision_score,
#     classification_report, confusion_matrix
# )
# # === 2. ROC Curve & AUC ===
# y_prob = log_reg.predict_proba(X_test)[:, 1]
# #
# fpr, tpr, thresholds = roc_curve(y_test, y_prob)
# roc_auc = roc_auc_score(y_test, y_prob)

# plt.figure(figsize=(6, 5))
# plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
# plt.plot([0, 1], [0, 1], 'k--', label="Random Guess")
# plt.xlabel("False Positive Rate")
# plt.ylabel("True Positive Rate (Recall)")
# plt.title("ROC Curve")
# plt.legend(loc="lower right")
# plt.show()

In [ ]:
# # === 3. Precision-Recall Curve ===
# precision, recall, pr_thresholds = precision_recall_curve(y_test, y_prob)
# ap_score = average_precision_score(y_test, y_prob)

# plt.figure(figsize=(6, 5))
# plt.plot(recall, precision, label=f'PR Curve (AP = {ap_score:.2f})')
# plt.xlabel("Recall")
# plt.ylabel("Precision")
# plt.title("Precision-Recall Curve")
# plt.legend(loc="upper right")
# plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import (
    roc_curve, roc_auc_score,
    precision_recall_curve, average_precision_score,
    classification_report, confusion_matrix
)
# Predicted probabilities for class 0 (defaulters)
y_scores = log_reg.predict_proba(X_test)[:, 0]

# ----------------------------
# 1. PR Curve Method
# ----------------------------
precision, recall, thresholds_pr = precision_recall_curve(y_test, y_scores, pos_label=0)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
best_idx_pr = np.argmax(f1_scores)
threshold_pr = thresholds_pr[best_idx_pr]

print("Optimal Threshold (PR Curve, max F1) for defaulters:", threshold_pr)
print("Precision:", precision[best_idx_pr])
print("Recall:", recall[best_idx_pr])
print("F1:", f1_scores[best_idx_pr])

# ----------------------------
# 2. ROC Curve Method
# ----------------------------
fpr, tpr, thresholds_roc = roc_curve(y_test, y_scores, pos_label=0)
j_scores = tpr - fpr
best_idx_roc = np.argmax(j_scores)
threshold_roc = thresholds_roc[best_idx_roc]

print("\nOptimal Threshold (ROC Curve, Youden's J) for defaulters:", threshold_roc)
print("TPR (Recall):", tpr[best_idx_roc])
print("FPR:", fpr[best_idx_roc])

# ----------------------------
# 3. Compare thresholds visually
# ----------------------------
plt.figure(figsize=(12,5))

# PR Curve
plt.subplot(1,2,1)
plt.plot(recall, precision, label='PR Curve')
plt.scatter(recall[best_idx_pr], precision[best_idx_pr], color='red', label='Optimal F1')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()

# ROC Curve
plt.subplot(1,2,2)
plt.plot(fpr, tpr, label='ROC Curve')
plt.scatter(fpr[best_idx_roc], tpr[best_idx_roc], color='red', label="Optimal J")
plt.plot([0,1],[0,1],'--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.title('ROC Curve')
plt.legend()

plt.show()

# ----------------------------
# 4. Predictions using PR optimal threshold
# ----------------------------
y_pred_optimal = (y_scores >= threshold_pr).astype(int)
cm = confusion_matrix(y_test, y_pred_optimal)
print("\nConfusion Matrix (PR Threshold):\n", cm)
print("\nClassification Report (PR Threshold):\n", classification_report(y_test, y_pred_optimal))